<a href="https://colab.research.google.com/github/YdkArda/ScienceAndArt/blob/main/week6_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit  bitsandbytes accelerate pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time # To simulate processing time

# --- 1. Model Configuration ---
@st.cache_resource
def load_mistral_model():
    """
    Loads the Mistral 7B Instruct model using 8-bit quantization.
    """
    model_id = "mistralai/Mistral-7B-Instruct-v0.2"

    # Colab GPU'su ile çalıştığından emin olmak için
    # Colab menüsünden Çalışma Zamanı -> Çalışma Zamanı Türünü Değiştir -> T4 GPU seçili olmalı!

    with st.spinner(f"Yapay Zeka Modeli Yükleniyor... ({model_id} - 8-bit Nicemleme)"):
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            load_in_8bit=True,
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model.eval()

    st.success("Model başarıyla yüklendi!")
    return model, tokenizer

# --- 2. Generation Function ---
def generate_response(model, tokenizer, prompt, max_len=1000):
    start_time = time.time()

    # Giriş token'larını modelin bulunduğu cihaza (GPU) taşı
    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **tokens,
            max_new_tokens=max_len,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(generated_ids[0].tolist(), skip_special_tokens=True)

    if full_output.startswith(prompt):
        response = full_output[len(prompt):].strip()
    else:
        response = full_output.strip()

    end_time = time.time()
    st.info(f"Yanıt {end_time - start_time:.2f} saniyede üretildi.")

    return response

# --- 3. Streamlit UI (Main App) ---
# Model yükleme sadece bir kez çalışır (@st.cache_resource sayesinde)
model, tokenizer = load_mistral_model()

# Set up the Streamlit Page
st.title("🤖 Mistral 7B Prompt Yanıtlama Uygulaması")
st.markdown("8-bit nicemleme (quantization) ile T4 GPU'da çalışan büyük dil modeli.")

user_prompt = st.text_area(
    "Lütfen metin isteminizi (prompt) girin:",
    value="Büyük dil modellerinin geleceği hakkında bir özet yaz.",
    height=200
)

if st.button("Yanıt Üret", type="primary"):
    if user_prompt:
        with st.status("Yanıt Üretiliyor...", expanded=True) as status:
            st.write("Model çıkarım yapıyor...")

            try:
                ai_response = generate_response(model, tokenizer, user_prompt)
                status.update(label="Yanıt Üretimi Tamamlandı!", state="complete", expanded=False)

                st.subheader("Mistral 7B Yanıtı:")
                st.markdown(ai_response)

            except Exception as e:
                st.error(f"Bir hata oluştu: {e}")
                status.update(label="Hata!", state="error")
    else:
        st.warning("Lütfen geçerli bir prompt girin.")

Writing app.py


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("36CNEZaIKezjKRQTzrD53pXEKPV_3qViyzHYqxZVA6W4ubgsi")
# Streamlit uygulamasını arka planda başlat
!streamlit run app.py &>/dev/null&

# Streamlit'in çalıştığı 8501 portuna tünel oluştur
public_url = ngrok.connect('8501')

# URL'yi yazdır
print(f"Uygulamanızın Genel URL'si: {public_url}")

Uygulamanızın Genel URL'si: NgrokTunnel: "https://semiannual-vida-livelily.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!nvidia-smi

Sun Nov 30 12:49:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             26W /   70W |    7578MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----